In [12]:
import pandas as pd
import json

# Load the JSON data
with open('games_ncaa/all_games.json', 'r') as file:
    games = json.load(file)

# Convert the JSON data to a pandas DataFrame
df_games_ncaa = pd.DataFrame(games)

In [13]:
df_games_ncaa

,date,home_team,away_team,home_score,away_score
0,2023-08-24,UAB,Northern Ky.,1,1
1,2023-08-24,Lindenwood,DePaul,0,1
2,2023-08-24,Wright St.,Xavier,1,1
3,2023-08-24,Mercer,Cal St. Fullerton,1,6
4,2023-08-24,Canisius,Saint Francis,1,2
...,...,...,...,...,...
1908,2023-12-02,Oregon St.,North Carolina,1,0
1909,2023-12-02,Stanford,Clemson,0,2
1910,2023-12-08,Clemson,West Virginia,1,0
1911,2023-12-08,Oregon St.,Notre Dame,0,1


In [14]:
# Load the JSON data
with open('games_wyscout/all_games_2023.json', 'r') as file:
    games = json.load(file)

# Convert the JSON data to a pandas DataFrame
df_games_wyscout = pd.DataFrame(games)

In [15]:
df_games_wyscout['match_date'] = pd.to_datetime(df_games_wyscout['match_date']).dt.strftime('%Y-%m-%d')
df_games_wyscout = df_games_wyscout[df_games_wyscout['match_date'] >= '2023-08-24']
df_games_wyscout = df_games_wyscout.sort_values(by='match_date')
df_games_wyscout

,wyId,match_date,home_team,away_team,home_score,away_score
41,5478926,2023-08-24,UMass Minutemen,Northeastern Huskies,2,2
1753,5516960,2023-08-24,Clemson Tigers,UCF Knights,1,2
604,5489105,2023-08-24,UNCW Seahawks,American Eagles,2,1
605,5489106,2023-08-24,North Carolina Heels,Loyola Greyhounds,1,1
606,5489107,2023-08-24,UNC Asheville Bulldogs,Navy Midshipmen,3,1
...,...,...,...,...,...,...
1060,5501578,2023-11-12,Stanford Cardinal,Berkeley Golden Bears,2,1
1480,5510852,2023-11-12,Santa Clara Broncos,Pacific Tigers,2,1
1479,5510851,2023-11-12,Portland Pilots,San Francisco Dons,2,0
1950,5542066,2023-11-12,VCU Rams,Dayton Flyers,1,2


In [16]:
ncaa_rename_dict = {
    "Adrian": "Adrian Bulldogs",
    "Air Force": "Air Force Falcons",
    "Akron": "Akron Zips",
    "American": "American Eagles",
    "Army West Point": "Army West Point Black Knights",
    "Assumption": "Assumption Greyhounds",
    "Aurora": "Aurora Spartans",
    "Averett": "Averett Cougars",
    "Bellarmine": "Bellarmine Knights",
    "Belmont": "Belmont Bruins",
    "Belmont Abbey": "Belmont Abbey Crusaders",
    "Binghamton": "Binghamton BearCats",
    "Blackburn": "(no pairing found)",
    "Boston College": "Boston College Eagles",
    "Boston U.": "Boston Terriers",
    "Bowling Green": "Bowling Green Falcons",
    "Bradley": "Bradley Braves",
    "Brandeis": "(no pairing found)",
    "Brown": "Brown Bears",
    "Bryant": "Bryant Bulldogs",
    "Bucknell": "Bucknell Bison",
    "Butler": "Butler Bulldogs",
    "CSU Bakersfield": "CSU Bakersfield Roadrunners",
    "CSUN": "CSUN Matadors",
    "Cal Poly": "Cal Poly Mustangs",
    "Cal St. Fullerton": "Cal State Fullerton Titans",
    "California": "(no pairing found)",
    "California Baptist": "California Baptist Lancers",
    "Campbell": "Campbell Camels",
    "Canisius": "Canisius Golden Griffins",
    "Central Ark.": "Central Arkansas Bears",
    "Central Conn. St.": "(no pairing found)",
    "Charlotte": "Charlotte 49ers",
    "Chicago St.": "Chicago State Cougars",
    "Clemson": "Clemson Tigers",
    "Cleveland St.": "Cleveland State Vikings",
    "Coastal Carolina": "Coastal Carolina Chanticleers",
    "Col. of Charleston": "Charleston Cougars",
    "Colgate": "Colgate Raiders",
    "Columbia": "Columbia Lions",
    "Cornell": "Cornell Big Red",
    "Creighton": "Creighton Bluejays",
    "Dartmouth": "Dartmouth Big Green",
    "Davidson": "Davidson Wildcats",
    "Dayton": "Dayton Flyers",
    "DePaul": "DePaul Blue Demons",
    "Delaware": "Delaware Blue Hens",
    "Denver": "Denver Pioneers",
    "Detroit Mercy": "Detroit Mercy Titans",
    "Drake": "Drake Bulldogs",
    "Drexel": "Drexel Dragons",
    "Duke": "Duke Blue Devils",
    "Duquesne": "Duquesne Duke",
    "ETSU": "ETSU Buccaneers",
    "Eastern Ill.": "Eastern Illinois Panthers",
    "Elon": "Elon Phoenix",
    "Embry-Riddle (FL)": "Embry Riddle Eagles",
    "Evansville": "Evansville Purple Aces",
    "FDU": "Fairleigh Dickinson Knights",
    "FGCU": "FGCU Eagles",
    "FIU": "FIU Panthers",
    "Fairfield": "Fairfield Stags",
    "Fla. Atlantic": "FAU Owls",
    "Fordham": "Fordham Rams",
    "Fresno Pacific": "Fresno Pacific Sunbird",
    "Furman": "Furman Paladins",
    "Ga. Southern": "Georgia Southern Eagles",
    "Gardner-Webb": "Gardner-Webb Bulldogs",
    "George Mason": "George Mason Patriots",
    "George Washington": "George Washington Colonials",
    "Georgetown": "Georgetown Hoyas",
    "Georgia St.": "Georgia State Panthers",
    "Gonzaga": "Gonzaga Bulldogs",
    "Grand Canyon": "Grand Canyon Lopes",
    "Green Bay": "Green Bay Phoenix",
    "Greensboro": "Greensboro Pride",
    "Hampden-Sydney": "Hampden-Sydney Tigers",
    "Harvard": "Harvard Crimson",
    "High Point": "High Point Panthers",
    "Hofstra": "Hofstra Pride",
    "Holy Cross": "Holy Cross Crusaders",
    "Houston Christian": "(no pairing found)",
    "Howard": "Howard Bison",
    "IU Indy": "(no pairing found)",
    "Indiana": "Indiana Hoosiers",
    "Iona": "Iona Gaels",
    "Jacksonville": "Jacksonville Dolphins",
    "James Madison": "JMU Dukes",
    "Kansas City": "(no pairing found)",
    "Kentucky": "Kentucky Wildcats",
    "LIU": "LIU Sharks",
    "LMU (CA)": "LMU Lions",
    "La Salle": "La Salle Explorers",
    "Lafayette": "Lafayette Leopards",
    "Le Moyne": "Le Moyne Dolphins",
    "Lehigh": "Lehigh Mountain Hawks",
    "Liberty": "Liberty Flames",
    "Lindenwood": "Lindenwood Lions",
    "Lipscomb": "Lipscomb Bisons",
    "Longwood": "Longwood Lancers",
    "Louisville": "Louisville Cardinals",
    "Loyola Chicago": "Loyola Chicago Ramblers",
    "Loyola Maryland": "Loyola Greyhounds",
    "Manhattan": "Manhattan Jaspers",
    "Marist": "Marist Red Foxes",
    "Marquette": "Marquette Golden Eagles",
    "Marshall": "Marshall Thundering Herd",
    "Mary Baldwin": "Mary Baldwin",
    "Maryland": "Maryland College Park Terrapins",
    "Massachusetts": "UMass Minutemen",
    "Memphis": "Memphis Tigers",
    "Mercer": "Mercer Bears",
    "Merrimack": "Merrimack Warriors",
    "Michigan": "Michigan Wolverines",
    "Michigan St.": "Michigan State Spartans",
    "Milwaukee": "Milwaukee Panthers",
    "Missouri St.": "Missouri State Bears",
    "Monmouth": "Monmouth Hawks",
    "Mount St. Mary's": "Mount St. Mary's Mountaineers",
    "NC State": "NC State Wolfpack",
    "NIU": "Northern Illinois Huskies",
    "NJIT": "NJIT Highlanders",
    "Navy": "Navy Midshipmen",
    "New Hampshire": "New Hampshire Wildcats",
    "Niagara": "Niagara Purple Eagles",
    "North Carolina": "North Carolina Heels",
    "North Florida": "North Florida Ospreyes",
    "Northeastern": "Northeastern Huskies",
    "Northern Ky.": "NKU Norse",
    "Northwestern": "Northwestern Wildcats",
    "Notre Dame": "Notre Dame Fighting Irish",
    "Oakland": "Oakland Golden Grizzlies",
    "Oberlin": "Oberlin Yeomen",
    "Ohio St.": "Ohio State Buckeyes",
    "Old Dominion": "Old Dominion Monarchs",
    "Omaha": "Omaha Mavericks",
    "Oral Roberts": "Oral Roberts Golden Eagles",
    "Oregon St.": "Oregon State Beavers",
    "Pacific": "Pacific Tigers",
    "Penn": "Penn Quakers",
    "Penn St.": "Penn State Nittany Lion",
    "Pittsburgh": "Pittsburgh Panthers",
    "Portland": "Portland Pilots",
    "Presbyterian": "Presbyterian BlueHose",
    "Princeton": "Princeton Tigers",
    "Providence": "Providence Friars",
    "Purdue Fort Wayne": "IPFW Mastodons",
    "Queens (NC)": "Queens Royals",
    "Quinnipiac": "Quinnipiac Bobcats",
    "Radford": "Radford Highlanders",
    "Rhode Island": "Rhode Island Rhody",
    "Rider": "Rider Broncs",
    "Robert Morris": "Robert Morris Colonials",
    "Rutgers": "Rutgers Scarlet Knights",
    "SIUE": "SIUE Cougars",
    "SMU": "SMU Mustangs",
    "SUNY Maritime": "(no pairing found)",
    "Sacramento St.": "Sacramento State Hornets",
    "Sacred Heart": "(no pairing found)",
    "Saint Francis": "(no pairing found)",
    "Saint Joseph's": "Saint Joseph's Hawks",
    "Saint Louis": "Saint Louis Billikens",
    "Saint Mary's (CA)": "St. Mary's College of CA Gaels",
    "Saint Peter's": "Saint Peter's Peacocks",
    "San Diego": "San Diego Toreros",
    "San Diego St.": "San Diego State Aztecs",
    "San Francisco": "San Francisco Dons",
    "San Jose St.": "San Jose State Spartans",
    "Santa Clara": "Santa Clara Broncos",
    "Seattle U": "Seattle Redhawks",
    "Seton Hall": "Seton Hall Pirates",
    "Siena": "Siena Saints",
    "South Carolina": "South Carolina Gamecocks",
    "South Fla.": "USF Bulls",
    "Southern Ind.": "Southern Indiana Screaming Eagles",
    "Southern Wesleyan": "Southern Wesleyan Warriors",
    "Southwest Baptist": "Southwest Baptist Bearcats",
    "St. Bonaventure": "St. Bonaventure Bonnies",
    "St. John's (NY)": "St John's Red Storm",
    "St. Joseph's (L.I.)": "Saint Joseph's Golden Eagles",
    "St. Thomas (MN)": "St. Thomas Tommies",
    "Stanford": "Stanford Cardinal",
    "Stetson": "Stetson Hatters",
    "Stonehill": "Stonehill College Skyhawks",
    "Stony Brook": "Stony Brook Seawolves",
    "Suffolk": "Suffolk Rams",
    "Syracuse": "Syracuse Orange",
    "Temple": "Temple Owls",
    "Trine": "Trine Thunder",
    "Trinity (TX)": "Trinity Tigers",
    "Tulsa": "Tulsa Hurricane",
    "UAB": "UAB Blazers",
    "UAlbany": "Albany Great Danes",
    "UC Davis": "UC Davis Aggies",
    "UC Irvine": "UC Irvine Anteaters",
    "UC Riverside": "UC Riverside Highlanders",
    "UCLA": "UCLA Bruins",
    "UMass Lowell": "UMass Lowell River Hawks",
    "UMKC": "UMKC Kangaroos",
    "UNC Asheville": "UNC Asheville Bulldogs",
    "UNC Greensboro": "UNC Greensboro Spartans",
    "UNC Wilmington": "UNC Wilmington Seahawks",
    "USC": "USC Trojans",
    "UTEP": "UTEP Miners",
    "Utah": "Utah Utes",
    "Utah Valley": "Utah Valley Wolverines",
    "Valparaiso": "Valparaiso Crusaders",
    "Vanderbilt": "Vanderbilt Commodores",
    "Vermont": "Vermont Catamounts",
    "Villanova": "Villanova Wildcats",
    "Virginia": "Virginia Cavaliers",
    "Virginia Tech": "Virginia Tech Hokies",
    "Wagner": "Wagner Seahawks",
    "Wake Forest": "Wake Forest Demon Deacons",
    "Washington": "Washington Huskies",
    "Washington St.": "Washington State Cougars",
    "West Virginia": "West Virginia Mountaineers",
    "Wichita St.": "Wichita State Shockers",
    "Wisconsin": "Wisconsin Badgers",
    "Wright St.": "Wright State Raiders",
    "Yale": "Yale Bulldogs"
}

df_games_ncaa['home_team'] = df_games_ncaa['home_team'].map(ncaa_rename_dict).fillna(df_games_ncaa['home_team'])
df_games_ncaa['away_team'] = df_games_ncaa['away_team'].map(ncaa_rename_dict).fillna(df_games_ncaa['away_team'])

In [17]:
df_games_ncaa

,date,home_team,away_team,home_score,away_score
0,2023-08-24,UAB Blazers,NKU Norse,1,1
1,2023-08-24,Lindenwood Lions,DePaul Blue Demons,0,1
2,2023-08-24,Wright State Raiders,Xavier,1,1
3,2023-08-24,Mercer Bears,Cal State Fullerton Titans,1,6
4,2023-08-24,Canisius Golden Griffins,(no pairing found),1,2
...,...,...,...,...,...
1908,2023-12-02,Oregon State Beavers,North Carolina Heels,1,0
1909,2023-12-02,Stanford Cardinal,Clemson Tigers,0,2
1910,2023-12-08,Clemson Tigers,West Virginia Mountaineers,1,0
1911,2023-12-08,Oregon State Beavers,Notre Dame Fighting Irish,0,1


In [18]:
df_games_wyscout

,wyId,match_date,home_team,away_team,home_score,away_score
41,5478926,2023-08-24,UMass Minutemen,Northeastern Huskies,2,2
1753,5516960,2023-08-24,Clemson Tigers,UCF Knights,1,2
604,5489105,2023-08-24,UNCW Seahawks,American Eagles,2,1
605,5489106,2023-08-24,North Carolina Heels,Loyola Greyhounds,1,1
606,5489107,2023-08-24,UNC Asheville Bulldogs,Navy Midshipmen,3,1
...,...,...,...,...,...,...
1060,5501578,2023-11-12,Stanford Cardinal,Berkeley Golden Bears,2,1
1480,5510852,2023-11-12,Santa Clara Broncos,Pacific Tigers,2,1
1479,5510851,2023-11-12,Portland Pilots,San Francisco Dons,2,0
1950,5542066,2023-11-12,VCU Rams,Dayton Flyers,1,2


In [19]:
df_games_wyscout[(df_games_wyscout['home_team'] == 'UAB Blazers') | (df_games_wyscout['away_team'] == 'UAB Blazers')]

,wyId,match_date,home_team,away_team,home_score,away_score
767,5491701,2023-08-24,NKU Norse,UAB Blazers,1,1
880,5497170,2023-08-28,Georgia State Panthers,UAB Blazers,5,1
965,5498128,2023-09-02,UAB Blazers,Chicago State Cougars,2,3
968,5498131,2023-09-03,UAB Blazers,HBU Huskies,1,1
977,5498142,2023-09-09,FGCU Eagles,UAB Blazers,2,1
920,5498031,2023-09-17,UAB Blazers,Charlotte 49ers,0,3
923,5498034,2023-09-23,Temple Owls,UAB Blazers,2,2
912,5497226,2023-09-27,UAB Blazers,Georgia Southern Eagles,1,1
929,5498044,2023-09-30,Tulsa Hurricane,UAB Blazers,1,0
930,5498045,2023-10-05,UAB Blazers,FIU Panthers,0,1


In [ ]:
# Open a text file to write the wyId of unmatched games
with open('unmatched_games.txt', 'w') as file:
    # Loop through each team in df_games_ncaa
    for team in df_games_ncaa['home_team'].unique():
        # Create separate dataframes for games with the team present in df_games_ncaa and df_games_wyscout
        df_ncaa_team_games = df_games_ncaa[(df_games_ncaa['home_team'] == team) | (df_games_ncaa['away_team'] == team)]
        df_wyscout_team_games = df_games_wyscout[(df_games_wyscout['home_team'] == team) | (df_games_wyscout['away_team'] == team)]
        
        # Compare their lengths
        if len(df_ncaa_team_games) == len(df_wyscout_team_games):
            print(f"Same length for {team}")
        else:
            print(f"Different length for {team}")
            for idx, wyscout_game in df_wyscout_team_games.iterrows():
                match_found = False
                for _, ncaa_game in df_ncaa_team_games.iterrows():
                    if (ncaa_game['home_team'] == wyscout_game['home_team'] and
                        ncaa_game['away_team'] == wyscout_game['away_team'] and
                        ncaa_game['home_score'] == wyscout_game['home_score'] and
                        ncaa_game['away_score'] == wyscout_game['away_score']) or \
                    (ncaa_game['home_team'] == wyscout_game['away_team'] and
                        ncaa_game['away_team'] == wyscout_game['home_team'] and
                        ncaa_game['home_score'] == wyscout_game['away_score'] and
                        ncaa_game['away_score'] == wyscout_game['home_score']):
                        match_found = True
                        break
                if not match_found:
                    print(f"Game not found in NCAA for {team}: {wyscout_game.to_dict()}")
                    file.write(f"{wyscout_game['wyId']}\n")

Same length for UAB Blazers
Same length for Lindenwood Lions
Same length for Wright State Raiders
Different length for Mercer Bears
Game not found in NCAA for Mercer Bears: {'wyId': 5511311, 'match_date': '2023-09-16', 'home_team': 'Winthrop Eagles', 'away_team': 'Mercer Bears', 'home_score': '0', 'away_score': '4'}
Game not found in NCAA for Mercer Bears: {'wyId': 5510711, 'match_date': '2023-09-30', 'home_team': 'Mercer Bears', 'away_team': 'VMI Keydets', 'home_score': '5', 'away_score': '1'}
Game not found in NCAA for Mercer Bears: {'wyId': 5510742, 'match_date': '2023-10-07', 'home_team': 'Wofford Terriers', 'away_team': 'Mercer Bears', 'home_score': '2', 'away_score': '0'}
Game not found in NCAA for Mercer Bears: {'wyId': 5484509, 'match_date': '2023-10-10', 'home_team': 'Davidson Wildcats', 'away_team': 'Mercer Bears', 'home_score': '0', 'away_score': '0'}
Game not found in NCAA for Mercer Bears: {'wyId': 5510762, 'match_date': '2023-10-21', 'home_team': 'UNCG Spartans', 'away_te